In [ ]:
import torch
import math
import nunmpy as np
from visdom import Visdom
import time
torch.__version__

### 使用Visdom在`PyTorch`中进行可视化

#### 安装

`pip install visdom`安装

`python -m visdom.server`在本地启动服务器,启动后会提示`It's Alive! You can navigate to http://localhost:8097`,我们打开浏览器，输入`http://localhost:8097`即可看到页面。

端口8097是默认的端口可以在启动命令后加`-port`参数指定端口，常用的参数还有`--hostname`,`-base_url`等。

#### 坑
Visdom的服务在启动时会自动下载一些静态文件，因为某些无法描述的原因，导致下载会失败，比如类似这样的提示`ERROR:root:Error 404 while downloading https://unpkg.com/layout-bin-packer@1.4.0`,就说明静态文件没有下载完全，这样有可能就会打不开或者页面中没有菜单栏，那么需要手动进行下载。

如果不知道conda的环境目录在哪里，可以使用`conda env list`查看。

### 基本概念
#### Environments
Environments的作用是对可视化区域进行分区，每个用户都会有一个叫做`main`的默认分区。在程序指定的情况下，默认的图表都会放到这里面。

#### Panes
`Panes`作为每一个可视化图表的容器，可以使用生成的图表，图片，文本进行填充，我们可以对`Panes`进行拖放，删除，调整大小和销毁等操作。

Panes和Environments是一对多的关系，即一个Environments可以包含多个Panes。

#### VIEW
在对Panes进行调整后，可以通过VIEW对状态进行管理：

#### 可视化接口
Visdom是由Plotly提供的可视化支持，所以提供以下可视化的接口:

- `vis.scatter`:2D 或 3D 散点图
- `vis.line`:线图
- `vis.stem`:茎叶图
- `vis.heatmap`:热力图
- `vis.bar`:条形图
- `vis.histogram`:直方图
- `vis.boxplot`: 箱型图
- `vis.surf`: 表面图
- `vis.contour`: 轮廓图
- `vis.quiver`: 绘出二维矢量场
- `vis.image`: 图片
- `vis.text`: 文本
- `vis.mesh`: 网格图
- `vis.save`: 序列化状态

### 使用
#### 绘制简单的图形

这里我们使用官方的DEMO来做样例




In [ ]:
env = Visdom()
assert env.check_connection()  #测试一下链接，链接错误的话会报错


In [ ]:
# 这里生成sin和cos两条曲线数据
Y = np.linspace(0, 2*math.pi, 70)
X = np.column_stack((np.sin(Y), np.cos(Y)))

In [ ]:
# 使用茎叶图展示
env.stem(
    X = X,
    Y = Y,
    opts = dict(legend = ['Sine', 'Cosine'])
)

可以通过env参数指定Environments，如果名称包含了下划线_那么visdom会跟根据下划线分割并自动分组

In [ ]:
envtest = Visdom(env = 'test_mesh')
assert envtest.check_connection()

生成一个网格图

In [ ]:
x = [0, 0, 1, 1, 0, 0, 1, 1]
y = [0, 1, 1, 0, 0, 1, 1, 0]
z = [0, 0, 0, 0, 1, 1, 1, 1]

X = np.c_[x, y, z]  # 将x,y,z组合成一个矩阵
i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2]
j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3]
k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6]
Y = np.c_[i, j, k]

envtest.mesh(
    X = X,
    Y = Y,
    opts = dict(opacity = 0.5)
)

### 更新损失函数
在训练的时候我们每一批次都会打印一下训练的损失和测试的准确率，这样展示的图表是需要动态增加数据的，下面我们来模拟一下这种情况

In [ ]:
x, y = 0, 0
env2 = Visdom()
pane1 = env2.line(
    X = np.array([x]),
    Y = np.array([y]),
    opts = dict(
        title = 'dynamic data'
    )
)

In [ ]:
for i in range(10):
    time.sleep(1) # 每隔一秒钟打印一次数据
    x += 1
    y = (y + i) * 1.5
    print(x, y)
    env2.line(
        X = np.array([x]),
        Y = np.array([y]),
        win = pane1,
        update = 'append'
    )

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvisison import models, datasets
torch.__version__


### 使用Tensorboard在 PyTorch 中进行可视化

#### Tensorboard 简介

Tensorboard是tensorflow内置的一个可视化工具，它通过将tensorflow程序输出的日志文件的信息可视化使得tensorflow程序的理解、调试和优化更加简单高效。

Tensorboard的可视化依赖于tensorflow程序运行输出的日志文件，因而tensorboard和tensorflow程序在不同的进程中运行。 

tensorboard虽然是tensorflow内置的可视化工具，但是他们跑在不同的进程中，所以Github上已经有大神将tensorboard应用到Pytorch中。


#### Tensorboard 安装

首先需要安装tensorboard

In [ ]:
pip install tensorboard

然后再安装tensorboardx 

In [ ]:
pip install tensorboardx

pytorch 1.1以后的版本内置了SummaryWriter 函数,所以不需要再安装tensorboardx了。

安装完成后与`visdom`一样执行独立的命令` tensorboard --logdir logs` 即可启动，默认的端口是 6006,在浏览器中打开 `http://localhost:6006/` 即可看到web页面。

这里要说明的是 微软的Edge浏览器css会无法加载，使用chrome正常显示。

### 页面
与visdom不同，tensorboard针对不同的类型人为的区分多个标签，每一个标签页面代表不同的类型。 

- SCALAR

对标量数据进行汇总和记录，通常用来可视化训练过程中随着迭代次数准确率(val acc)、损失值(train/test loss)、学习率(learning rate)、每一层的权重和偏置的统计量(mean、std、max/min)等的变化曲线。

- IMAGES

可视化当前轮训练使用的训练/测试图片或者 feature maps

- GRAPHS

可视化计算图的结构及计算图上的信息，通常用来展示网络的结构

- HISTOGRAMS

可视化张量的取值分布，记录变量的直方图（统计张量随着迭代轮数的变化情况）

- PROJECTOR

全称Embedding Projector高维向量进行可视化

### 使用
在使用前请先去确认执行`tensorboard --logdir logs`并保证`http://localhost:6006/`页面能够正常打开

#### 图像展示

首先介绍比较简单的功能，查看我们训练集和数据集中的图像，这里我们使用现成的图像作为展示。这里使用wikipedia上的一张猫的图片

In [ ]:
#引入 tensorboardX 包
#  这里的引用也要修改成torch的引用
from torch.utils.tensorboard import SummaryWriter

In [ ]:
cat_img = Image.open('./....')
cat_img.size

这是一张1280x853的图，我们先把她变成224x224的图片，因为后面要使用的是vgg16

In [ ]:
transform_224 = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

cat_img_224 = transform_224(cat_img)

将图片展示在tebsorboard中：

In [ ]:
writer = SummaryWriter(log_dir = './logs', comment = 'cat image')  # 这里的logs要与--logdir的参数一样
writer.add_image("cat",cat_img_224)
writer.close()# 执行close立即刷新，否则将每120秒自动刷新

浏览器访问[[http://localhost:6006/#images]]即可看到猫的图片

#### 更新损失函数
更新损失函数和训练批次我们与visdom一样使用模拟展示，这里用到的是tensorboard的SCALAR页面

In [ ]:
x = torch.FloatTensor([100])
y = torch.Floattensor([500])

for epoch in range(30):
    x = x * 1.2
    y = y / 1.1
    loss = np.random.random()
    with SummaryWriter(log_dir = './logs', comment = 'train') as writer:
        writer.add_histogram('his/x', x, epoch)
        writer.add_histogram('his/y', y, epoch)
        writer.add_scalar('data/x', x, epoch)
        writer.add_scalar('data/y', y, epoch)
        writer.add_scalar('data/loss', loss, epoch)
        writer.add_sclars('data/data_group', {'x': x, 'y': y}, epoch)

浏览器访问[http://localhost:6006/#scalars]即可看到图形

#### 使用PROJECTOR对高维向量可视化

PROJECTOR的的原理是通过PCA，T-SNE等方法将高维向量投影到三维坐标系（降维度）。Embedding Projector从模型运行过程中保存的checkpoint文件中读取数据，默认使用主成分分析法（PCA）将高维数据投影到3D空间中，也可以通过设置设置选择T-SNE投影方法，这里做一个简单的展示。

In [ ]:
BATCH_SIZE = 512
EPOCHS = 20
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train = True, download = True,
                   transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.137,), (0.3081,))
                   ])),
    batch_size = BATCH_SIZE, shuffle = True
)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(20*10*10, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        in_size = x.size(0)
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)
        out = self.conv2(out)
        out = F.relu(out)
        out = out.view(in_size, -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out, dim = 1)
        return out

model = ConvNet()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train(model, train_loader, optimizer, epoch):
    n_iter = 0
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 30 == 0:
            n_iter = n_iter + 1
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

            # 相较于以前的训练方法，主要增加了如下内容
            out = torch.cat((output.data, torch.ones(len(output), 1)), 1)
            with SummaryWriter(log_dir='./logs', comment='mnist') as writer: 
                #使用add_embedding方法进行可视化展示
                writer.add_embedding(
                    out,
                    metadata = target.data,
                    label_img = data.data,
                    global_step = n_iter
                )

这里节省时间，只训练一次

In [ ]:
train(model, train_loader, optimizer, 0)

打开[http://localhost:6006/#projector]即可看到效果。

#### 绘制网络结构
在pytorch中我们可以使用print直接打印出网络的结构，但是这种方法可视化效果不好，这里使用tensorboard的GRAPHS来实现网络结构的可视化。 由于pytorch使用的是动态图计算，所以我们这里要手动进行一次前向的传播。

使用Pytorch已经构建好的模型进行展示



In [ ]:
vgg16 = models.vgg16(pretrained = True)
print(vgg16)

在前向传播前，先要把图片做一些调整

In [ ]:
transform_2 = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

使用上一张猫的图片进行前向传播

In [ ]:
vgg16_input = transform_2(cat_img)[np.newaxis] # 因为pytorch的是分批次进行的，所以我们这里建立一个批次为1的数据集
vgg16_input.shape

开始前向传播，打印输出值

In [ ]:
out = vgg16(vgg16_input)
_, preds = torch.max(out.data, 1)
label = preds.numpy()[0]
label

将结构图在tensorboard进行展示

In [ ]:
with SummaryWriter(log_dir = './logs', comment = 'vgg161') as writer:
    writer.add_graph(vgg16, vgg16_input)

对于Pytorch的1.3版本来说，实测 SummaryWriter在处理结构图的时候是有问题的（或者是需要加什么参数，目前我还没找到），所以建议大家继续使用tensorboardx。

### 可视化理解卷积神经网络

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
cat_img=Image.open('./1280px-Felis_silvestris_catus_lying_on_rice_straw.jpg')
transform_224= transforms.Compose([
    transforms.Resize(224), 
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])


cat_img_224=transform_224(cat_img)

上面的代码是我们读取了一张图片，并对图片进行了一些预处理，下面我们来创建vgg16的预训练好网络模型

In [ ]:
net = models.vgg16(pretrained = True)
inputs = cat_img_224[np.newaxis] #这两个方法都可以cat_img_224[None,::]

进行一次前向的传播，看看得到了什么结果

In [ ]:
out = net(inputs)
_, preds = torch.max(out.data, 1)
preds
label = preds.numpy()[0]
label

不同的预训练权重也会出现不同的结果

#### 卷积神经网络的可视化背景
CNN模型虽然在图像处理上表现出非常良好的性能和准确性，但一直以来都被认为是一个黑盒模型，人们无法了解里面的工作机制。 针对这个问题，研究人员除了从理论层面去寻找解释外，也提出了一些可视化的方法直观地理解CNN的内部机理。

#### 基于Deconvolution的方法
Visualizing and Understanding Convolutional Networks 主要是将激活函数的特征映射回像素空间，来揭示什么样的输入模式能够产生特定的输出,因为网络是有层级关系的，所以越靠近输出的层级学到的特征越抽象，与实际任务越相关

#### 基于Backpropagation的方法


In [ ]:
import sys
sys.path.insert(0, './src/')

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def rescale_grads(map, gradtype = "all"):
    if(gradtype == "pos"):
        map = (np.maximum(0, map) / map.max())
    elif gradtype == "neg":
        map = (np.maximum(0, -map)/ -map.min())
    else:
        map = map - map.min()
        map /= map.max()

    return map

#### Guided-Backpropagation
大致的方法为：选择某一种输出模式，然后通过反向传播计算输出对输入的梯度。这种方式与上一种deconvnet的方式的唯一区别在于对ReLU梯度的处理。

ReLU在反向传播的计算采用的前向传播的特征作为门阀，而deconvnet采用的是梯度值，guided-backpropagation则将两者组合在一起使用，这样有助于得到的重构都是正数。

In [ ]:
inputs.requires_grad = True   # 这句话必须要有，否则会报错
from guided_backprop import GuidedBackprop    #这里直接引用写好的方法，在src，目录找想对应的文件
GB = GuidedBackprop(net)
gp_grads = GB.generate_gradients(inputs, label)
gp_grads = np.moveaxis(gp.grads, 0, -1)

# 分别计算三类gp
ag = rescale_grads(gp_grads, gradtype = "all")
pg = rescale_grads(gp_grads, gradtype = "pos")
ng = rescale_grads(gp_grads, gradtype = "neg")


In [ ]:
# 使用matplotlib查看结果
plt.imshow(cat_img)

In [ ]:
plt.imshow(ag)

In [ ]:
plt.imshow(ng)

上面三张图是rbg三个通道的展示结果，下面我们合并成一个通道再看一下

In [ ]:
gag = rgb2gray(ag)
plt.imshow()

In [ ]:
gpg = rgb2gary(pg)
plt.imshow()

In [ ]:
gng = rgb2gray(ng)
plt.imshow()

####  CAM（Class Activation Map）
这个方法严格来说不是基于梯度的，但是后面我们会将反向传播与CAM整合，所以简单的对CAM做个说明。

CAM（class activation map）是指输入中的什么区域能够指示CNN进行正确的识别。

通常特征图上每个位置的值在存在其感知野里面某种模式时被激活，最后的class activation map是这些模式的线性组合，我们可以通过上采样，将class activation map 还原到与原图一样的大小，通过叠加，我们就可以知道哪些区域是与最后分类结果息息相关的部分。


#### Grad-CAM
顾名思义 Grad-CAM的加权系数是通过反向传播得到的，而CAM的特征加权系数是分类器的权值。

Grad-CAM 与 CAM相比，它的优点是适用的范围更广，Grad-CAM对各类结构，各种任务都可以使用。这两种方法也可以应用于进行弱监督下的目标检测，后续也有相关工作基于它们进行改进来做弱监督目标检测。

In [ ]:
import math
from gradcam import GradCam
from guided_gradcam import guided_grad_cam
from guided_backprop import GuidedBackprop
nlayers = len(net.features._modules.item()) - 1 
print(nlayers)
cam_list = []

for layer in range(nlayers):
    # GradCam
    grad_cam = GradCam(net, target_layer = layer)
    cam = grad_cam.generate_cam(inputs, label)

    # GuidedBackprop
    GBP = GuidedBackprop(net)
    guided_grads = GBP.generate_gradients(inputs, label)

    # Guided Grad Cam
    cam_gb = guided_grad_cam(cam, guided_grads)
    cam_list.append(rgb2gray(np.moveaxis(cam_gb, 0, -1)))


In [ ]:
plt.imshow(cam_list[0])

需要注意的是，在使用 Visualizing and Understanding Convolutional Networks的时候，对网络模型是有要求的，要求网络将模型包含名为features的组合层，这部分是代码中写死的，所以在pytorch的内置模型中，vgg、alexnet、densenet、squeezenet是可以直接使用的，inception(googlenet)和resnet没有名为features的组合层，如果要使用的话是需要对代码进行修改的。